In [1]:
from notebook.utils import url_path_join
from notebook.base.handlers import IPythonHandler
class HelloWorldHandler(IPythonHandler):
    def get(self):
        self.finish('Hello, world!')
        
def load_jupyter_server_extension(nb_server_app):
    """
    Called when the extension is loaded.
    Args:
    nb_server_app (NotebookWebApplication): handle to the Notebook webserver
    ˓→instance.
    """
    web_app = nb_server_app.web_app
    host_pattern = '.*$'
    route_pattern = url_path_join(web_app.settings['base_url'], '/hello')
    web_app.add_handlers(host_pattern, [(route_pattern, HelloWorldHandler)])


In [2]:
import os
import json
from IPython.display import HTML

def run():
    exclude = set(['.ipynb_checkpoints'])
    metaFiles = {}
    for root, dirs, files in os.walk(os.path.realpath('.'), topdown=True):
        dirs[:] = [d for d in dirs if d not in exclude]
        for file in files:
            if file.endswith('.ipynb'):
                path = os.path.join(root, file)
                with open(path, 'r') as f:
                    data = json.load(f)
                    try:
                        if 'info' in data['metadata']:
                            metaFiles[path] = data['metadata']['info']
                        else:
                            metaFiles[path] = {}
                    except:
                        print("Error: %s has no metadata" % path)
                    
    return metaFiles
#%timeit run()
meta = run()
meta['__path__'] = os.path.realpath('.')

In [3]:
def sendComm(comm, msg):
    @comm.on_msg
    def _recv(msg):
        print(msg)
    comm.send(meta)

comm = get_ipython().kernel.comm_manager.register_target('meta', sendComm)

In [4]:
display = '''<!DOCTYPE html>
<html>
<head>
<meta name="viewport" content="width=device-width, initial-scale=1">
<style>
* {
  box-sizing: border-box;
}

#myInput {
  background-image: url('/css/searchicon.png');
  background-position: 10px 10px;
  background-repeat: no-repeat;
  width: 100%;
  font-size: 16px;
  padding: 12px 20px 12px 40px;
  border: 1px solid #ddd;
  margin-bottom: 12px;
}

#NotebookPathsTable {
  border-collapse: collapse;
  width: 100%;
  border: 1px solid #ddd;
  font-size: 18px;
}

#NotebookPathsTable th, #NotebookPathsTable td {
  text-align: left;
  padding: 12px;
}

#NotebookPathsTable tr {
  border-bottom: 1px solid #ddd;
}

#NotebookPathsTable tr.header, #NotebookPathsTable tr:hover {
  background-color: #f1f1f1;
}
</style>
</head>
<body>

<input type="text" id="myInput" onkeyup="myFunction()" placeholder="Search for notebooks.." title="Type in a name">

<script>
function isEmpty(obj) {
  return Object.keys(obj).length === 0;
}

function buildTable()
{
    comm = Jupyter.notebook.kernel.comm_manager.new_comm('meta');
    var table = document.getElementById('NotebookPathsTable');
    
    
    // Create the table header
    var tbody = document.createElement('tbody');
    var tr = document.createElement('tr');
    tr.className = 'header';
    
    var th = document.createElement('th');
    var text = document.createTextNode('Notebook');
    th.style = 'width:60%;';
    th.appendChild(text);
    tr.appendChild(th);
    
    var th = document.createElement('th');
    var text = document.createTextNode('Grade');
    th.style = 'width:20%;';
    th.appendChild(text);
    tr.appendChild(th);
    
    var th = document.createElement('th');
    var text = document.createTextNode('Subject');
    th.style = 'width:20%;';
    th.appendChild(text);
    tr.appendChild(th);

    tbody.appendChild(tr);
    table.appendChild(tbody);
    
    var tableData;
    
    // Register a handler
    comm.on_msg(function(msg) {
        tableData = msg.content.data;
        
        var path = encodeURI(tableData['__path__']).split('/');
        var url = document.URL.split('/');
        
        // Finds what is not in common between the url and the path
        var pathPrefix = '';
        var urlPrefix  = '';
        for(var i = url.length-1; url.length-1-i < path.length; --i)
        {
            if(url[i-1] != path[i-(url.length-path.length)])
            {
                console.log(url[i-1]);
                console.log(path[i-(url.length-path.length)]);

                // j <= i-1 to include the difference
                for(var j = 0; j <= i-1; ++j)
                    urlPrefix += url[j] + '/';
                for(var j = 0; j <= i-(url.length-path.length); ++j)
                    pathPrefix += path[j] + '/';
                    
                console.log(pathPrefix);
                console.log(urlPrefix);
                break;
            }
        }
        
        console.log(path)
        console.log(url)
        
        for(var key in tableData) 
        {
            if(tableData.hasOwnProperty(key)) 
            { 
                // Filter out python hidden elements
                if(!key.startsWith('__'))
                {
                    if(!isEmpty(tableData[key]))
                    {
                        // Convert the key from a disk path to a url path
                        urlPath = encodeURI(urlPrefix) + 
                                  encodeURI(key.slice(pathPrefix.length, key.length));
                        console.log(key, tableData[key]);
                        
                        var tr = document.createElement('tr');
                        var columns = ['Grade', 'Subject']
                        
                        
                        var td = document.createElement('td');
                        var a = document.createElement('a');
                        var text = document.createTextNode('&nbsp;');
                        a.appendChild(text);
                        a.href = urlPath;
                        a.style = 'display:block';
                        td.appendChild(a);

                        var notebookName = urlPath.split('/').pop().split('.');
                        notebookName = decodeURI(notebookName.slice(0, notebookName.length-1));
                        var text = document.createTextNode(notebookName);
                        td.appendChild(text);
                        //tr.href = urlPath;
                        tr.appendChild(td);
                        
                        for(var i = 0; i < columns.length; ++i)
                        {
                            td = document.createElement('td');
                            var text = document.createTextNode(tableData[key][columns[i]]);
                            td.appendChild(text);
                            tr.appendChild(td);  
                        }
                        
                        table.appendChild(tr);
                    }
                }
            }
        }
    });
}
buildTable();
</script>


<table id='NotebookPathsTable'>
</table>

<script>
function myFunction() {
  var input, filter, table, tr, td, i;
  input = document.getElementById("myInput");
  filter = input.value.toUpperCase();
  table = document.getElementById("NotebookPathsTable");
  tr = table.getElementsByTagName("tr");
  for (i = 0; i < tr.length; i++) {
    td = tr[i].getElementsByTagName("td")[0];
    if (td) {
      if (td.innerHTML.toUpperCase().indexOf(filter) > -1) {
        tr[i].style.display = "";
      } else {
        tr[i].style.display = "none";
      }
    }       
  }
}
</script>

</body>
</html>'''

In [5]:
HTML(display)